In [3]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio as rio
from numba import jit
from rasterio.merge import merge
from globalstack_functions import h_flowdir, pq, sinkfill, stack_bilinear
%load_ext autoreload
%autoreload 2

## For custom flow routing.  This section prepares the grids for what I called the 'd8' flow routing method based on pit-filled grids created using the create_pit_filled.py scipt. Skip this step if using HydroSHEDS.  Otherwise, you will need to run create_pit_filled.py first, which takes the longest by far.  Ideally you would run this script on a computer with a lot of memory.  

In [ ]:
Z = np.load('/Volumes/Samsung_T5/eu/others/dem.npy',mmap_mode='r')#[20000:25000,20000:25000]

In [ ]:
sx, sy = slp(Z)

In [ ]:
np.save('/Volumes/Samsung_T5/stack_rx',sx)

In [ ]:
np.save('/Volumes/Samsung_T5//stack_ry',sy)

In [ ]:
del(sx)
del(sy)

In [21]:
np.max(Z-a)

0.6430145801697291

## This nb generates the grids used in RiverProfileApp

### Download and unzip all GRID data from Hydrosheds Copy the cell below into the QGIS python console to load. Use qgis to merge the tiles into a 'dir.tif' grid with 16 bit int

In [ ]:
#Qgis add all lyr

import os
dir1 = '/Users/gr_1/Documents/DEM_3s_GRID/as_dem_3s_zip_grid///'
n1=0
for nm in os.listdir(dir1): 
    if not('.' in nm):
        fs = os.listdir(dir1+nm)
        for f in fs:
            if not '.' in f:
                iface.addRasterLayer(dir1+'/'+nm+'/'+f)

### Prepare the receiver grids - load in the direction grid

##And open the tif with the drainage directions 

In [ ]:
f = rio.open('/Volumes/Samsung_T5/au/')
a_sub=f.read()
a_sub = np.uint16(a_sub.squeeze()) # Need to squeeze to remove extra dimension
a_sub = zero_edges(a_sub)

In [ ]:
lb = f.bounds.left  # These bounds are used by RPA, we input them into the global variables at the beginning of profiler_app.py
ub = f.bounds.top

In [ ]:
## Now we convert hydrosheds drainage direction to x and y

In [ ]:
sx, sy = h_flowdir(a_sub);

In [ ]:
np.save('/Volumes/T7/stack_rx2.npy',sx)
np.save('/Volumes/T7/stack_ry2.npy',sy)

In [ ]:
del(sy)## Considerable memory allocation may require deletion, then we reload as memory mapped grids
del(sx)
del(a_sub)

## reload as memory mapped grids

In [ ]:
sx = np.load('/Volumes/{}/{}/rs/stack_rx.npy'.format(disk,folder),mmap_mode='r')
sy = np.load('/Volumes/{}/{}/rs/stack_ry.npy'.format(disk,folder),mmap_mode='r')

In [ ]:
#np.load('/Volumes/Samsung_T5/Af_ac.tif.npy', mmap_mode='r')

## Now we caluclate the stack 

In [ ]:
Iy, Ix = stack_bilinear(sx, sy)

In [ ]:
np.save('/Volumes/{}/{}/sa_Iy'.format(disk,folder), Iy)

In [ ]:
np.save('/Volumes/{}/{}/sa_IX'.format(disk,folder), Ix)

In [ ]:
del(Ix)

In [ ]:
del(Iy)

## The beginning of acc caluclation

In [ ]:
size = (96000, 94800) # Africa
size = (84000, 232800) # Asia
size = np.shape(sx)

In [ ]:
Ix = np.load('/Volumes/Samsung_T5/sa/sa_Ix.npy', mmap_mode='r')
Iy = np.load('/Volumes/Samsung_T5/sa/sa_Iy.npy', mmap_mode='r')
#sx = np.load('/Volumes/T7/stack_rx.npy', mmap_mode='r')
#sy = np.load('/Volumes/T7/stack_ry.npy', mmap_mode='r')

In [ ]:
#size = (96000, 94800)
A = acc(sx, sy, Ix, Iy, size)

In [ ]:
np.save('/Volumes/Samsung_S5/na/acc', A)

In [ ]:
del(A)

In [ ]:
import geopandas as gpd

In [ ]:
a = gpd.read_file('/Users/gr_1/Documents/streamprofiler/af_bound')

In [ ]:
a.to_file('/Users/gr_1/Documents/af',driver='GeoJSON')

In [ ]:
str(a.to_json())

## Brief Test

## Erates

In [ ]:
disk = 'Samsung_T5'
folder = 'na'
Ix = np.load('/Volumes/{}/{}/sa_Ix.npy'.format(disk,folder), mmap_mode='r')
Iy = np.load('/Volumes/{}/{}/sa_Iy.npy'.format(disk,folder), mmap_mode='r')
#
sx = np.load('/Volumes/{}/{}/rs/stack_rx.npy'.format(disk,folder),mmap_mode='r')
sy = np.load('/Volumes/{}/{}/rs/stack_ry.npy'.format(disk,folder),mmap_mode='r')
Acc = np.load('/Volumes/{}/{}/others/acc.npy'.format(disk,folder),mmap_mode='r')
Z = np.load('/Volumes/{}/{}/others/dem.npy'.format(disk,folder), mmap_mode='r')
#Ef = np.load('/Volumes/{}/{}/Ef.npy'.format(disk,folder), mmap_mode='r')

#A = np.load('/Volumes/T7/zipped/eu/ot')

In [ ]:
## Condition DEM??
@jit(nopython=True)
def condition():
    Zn = np.copy(Z)
    for i in range(len(Ix)):
        ry = Iy[i] + sy[Iy[i], Ix[i]]
        rx = Ix[i] + sx[Iy[i], Ix[i]]
        if Zn[0,Iy[i], Ix[i]] <= Zn[0,ry, rx] and Zn[0,ry,rx] >0:
            Zn[0,Iy[i], Ix[i]] = Zn[0,ry,rx]+1e-8
        if np.mod(i,1e6) ==0:
            print(i/len(Ix))
    return Zn

In [ ]:
Zn = condition()

In [ ]:
Z = np.save('/Volumes/{}/{}//Zn.npy'.format(disk,folder),Zn)

In [ ]:
Z = np.load('/Volumes/{}/{}//Zn.npy'.format(disk,folder),mmap_mode='r')

In [ ]:
E = np.zeros(np.shape(Acc),dtype=np.float32)

In [ ]:
erat = np
@jit(nopython=True)

def ero_f(E):
    k = 1e-6
    n = 2.06
    m = n / 2
    maxs=0
    count = 0
    #D / k = 4493
    summ = 0
    
    ny,nx = np.shape(E)
    for i in range(int((len(Ix)))):
        ry = Iy[i] + sy[Iy[i], Ix[i]]
        rx = Ix[i] + sx[Iy[i], Ix[i]]
        if (Acc[Iy[i], Ix[i]] <=12 or Ix[i] < 3 or Iy[i] < 3 or Iy[i] > ny-3 or Ix[i] > nx -3):
            continue
        lat = 60.0 - Iy[i] * 1/1200

        dx = np.cos(np.abs(lat) / 180 * np.pi) * (1852/60) * 3
        dy = 92.6
        dl = np.sqrt(((Iy[i] - ry) * dy) ** 2.0 + ((Ix[i] - rx) * dx)**2 )
        fact = (dx*dy)**m * k

        if dl ==0:
            continue
        s = (Z[0,Iy[i], Ix[i]] - Z[0,ry, rx]) / (dl)
        u = 1
        if s < 0:
            u = -1
            s = -s
        #s = s / 2 + np.abs(s) / 2
       # print(k * Acc[Iy[i], Ix[i]]**m * (dx*dy)**m * s**n)
        e = u* fact * Acc[Iy[i], Ix[i]] ** m * s ** n
        if e>2000:
            e = 2000
        if e<-2000:
            e = -2000
        if e > maxs:
            maxs = e
            #print(Acc[Iy[i], Ix[i]])
            #print(s)
        E[Iy[i], Ix[i]] = e
        summ += e #E[Iy[i], Ix[i]]
        if np.mod(i, 1000000) == 0:
            print(i/len(Ix))
            print(summ)
            #print(count)
    return E

In [ ]:
E = ero_f(E)

In [ ]:
np.save('/Volumes/{}/{}//Ef.npy'.format(disk,folder),E)

In [ ]:
e2[np.isnan(e2)] = 0

In [ ]:
plt.imshow(e2,vmin=-.1,vmax=.1)

##### e[10000:15000,10000:15000]

In [ ]:
ny, nx = np.shape(sx)
E = np.zeros((ny, nx),dtype=np.float32)

In [ ]:
@jit(nopython=True)
def diffuse(E, D, dt):
    ny, nx = np.shape(sy)
    dy = 92.6
    e=0
    summ =0
    for i in range(1, ny - 1):
        lat = 60.0 - i * 1/1200  ### set to top lognitude
        dx = np.cos(np.abs(lat) / 180 * np.pi) * (1852/60) * 3
        print(dx)
        for j in range(1, nx - 1):
            e= D * ((2 * Z[0, i, j] -
                            Z[0, i + 1, j] -
                            Z[0, i - 1, j]) /
                           (dy ** 2) +
                           (2 * Z[0, i, j] - Z[0, i, j + 1] - Z[0, i, j - 1])
                           / (dx ** 2))
            E[i, j] = e
            summ += (np.abs(e) + e)/2
            if j%50000 == 0:
                print(i/ny)
                print(summ)
            
    E *= dt
    print(summ)
    return E

In [ ]:
E = diffuse(E, 1e-6*4552, 1)

In [ ]:
np.save('/Volumes/{}/{}//E.npy'.format(disk,folder), E)

In [ ]:
@jit(nopython=True)
def calcsum():
    summ = 0
    ny, nx = np.shape(e1)
    for i in range(1, ny - 1):
        for j in range(1, nx - 1):
            summ += (np.abs(e1[i, j]) + e1[i, j])/2 + e2[i, j]
    return(summ)
summ = calcsum()

In [ ]:
@jit(nopython=True)
def calcsum():
    summ = 0
    ny, nx = np.shape(e1)
    for i in range(1, ny - 1):
        for j in range(1, nx - 1):
            summ += (np.abs(e2[i, j]) + e2[i, j])/2 
    return(summ)

In [ ]:
e2 = np.load('/Volumes/{}/{}/e.npy'.format(disk,folder),mmap_mode='r')

In [ ]:
#elev
@jit()
def ep():
    ny, nx = np.shape(e2)
    sum0=0
    sum1=0
    sum2=0
    sum3=0
    sum4=0
    n0=0
    n1=0
    n2=0
    n3=0
    n4=0
    dy = 92.5
    for i in range(ny):
        lat = 60.0 - i * 1/1200  ### set to top lognitude
        dx = np.cos(np.abs(lat) / 180 * np.pi) * (1852/60) * 3
        #print(dx)
        for j in range(nx):
            er = e2[i, j]*dx*dy
            if er<0:
                er=0

            
            z = Z[0, i, j]
            if z<0:
                continue
            if z > 0 and z <= 500:
                sum0 += er
                n0 +=1
            if z > 500 and z <= 1000:
                sum1 += er
                n1+=1
            if z > 1000 and z <= 2500:
                sum2 += er
                n2+=1
            if z > 2500 and z <= 4000:
                sum3 += er
                n3+=1
            if z> 4000:
                sum4 += er
                n4+=1
    return [sum0, sum1, sum2, sum3, sum4]
l = ep()

In [ ]:
#slope
@jit()
def ep():
    ny, nx = np.shape(e2)
    sum0=0
    sum1=0
    sum2=0
    sum3=0
    sum4=0
    n0=0
    n1=0
    n2=0
    n3=0
    n4=0
    dy = 92.5
    for i in range(ny):
        lat = 60.0 - i * 1/1200  ### set to top lognitude
        dx = np.cos(np.abs(lat) / 180 * np.pi) * (1852/60) * 3
        dy = 92.6
        
        #print(dx)
        
        for j in range(nx):
            er = e2[i, j]*dx*dy
            i2 = sy[i,j] + i
            j2 = sx[i,j] + j
            dist = np.sqrt(((i - i2)*dy)**2 + ((j-j2)*dx)**2)
            
            if er<0:
                er=0

            
            z = Z[0, i, j]
            z2 = Z[0, i2, j2]
            s = (z-z2)/dx
            if s >= 0 and s <= .1:
                sum0 += er
                n0 +=1
            if s > .1 and s <= 1:
                sum2 += er
                n2+=1
            if s > 1:
                sum3 += er
                n3+=1
   
    return [sum0, sum1, sum2, sum3, sum4], [n0,n1,n2,n3,n4]
l = ep()

In [ ]:
l

In [ ]:
l1s = []
l2s = []

In [ ]:
(sum(l1) +sum(l2)) * .51 * 2.3 /1e12 * 2400/100

In [ ]:
#Asia
l1 = [35606913966.23573, 0.0, 180764528827.63324, 26513995568.252655, 0.0]
 #[16388669297, 0, 2531939123, 94760001, 0])

l2 = [32862239142.32389, 0.0, 108664402731.8776, 16960568645.82187, 0.0]
# [16388669297, 0, 2531939123, 94760001, 0])
l1 = np.array(l1)
l2 = np.array(l2)
l2s.append(l2)
l1s.append(l1)

In [ ]:
#Aus 
l1 = [3868802281.3529835, 0.0, 4959187705.169707, 293735229.85617614, 0.0]
# [4222754220, 0, 146189582, 2586137, 0])

l2 = [8082546.014218119, 0.0, 6648300.472072207, 482836.5276691823, 0.0]
# [4222754220, 0, 146189582, 2586137, 0])
l1 = np.array(l1)
l2 = np.array(l2)
l2s.append(l2* 1000)
l1s.append(l1) 

In [ ]:
#Af
l1 =    [34449409612.48713, 0.0, 27740889432.914272, 6882989346.678329, 0.0]
  #   [7404345658, 0, 578759449, 7605091, 0])
l2 =   [39488366.428432055, 0.0, 29485494.850450497, 1646821.374632989, 0.0]
     #[7404345658, 0, 578759449, 7605091, 0])
l1 = np.array(l1)
l2 = np.array(l2)
l2s.append(l2)
l1s.append(l1)

In [ ]:
sum(l2)/sum(l1)

In [ ]:
#Na
l1 = [13731378198.030537, 0.0, 46065644307.39387, 4205185143.059032, 0.0]
 #[6311155310, 0, 811488921, 34020891, 0])


# [6311155310, 0, 811488921, 34020891, 0])
l1 = np.array(l1)
l2 = np.array(l2)
l2s.append(l2* 1000) 
l1s.append(l1)


In [ ]:
#Sa
l1 = [12362150746.902735, 0.0, 56575070812.39045, 8192751360.001698, 0.0]
 #[5316456410, 0, 692078079, 23819971, 0])


l2 =[13264732.166475572, 0.0, 34582520.14444802, 4962060.7614813605, 0.0]
# [5316456410, 0, 692078079, 23819971, 0])

l1 = np.array(l1)
l2 = np.array(l2)
l2s.append(l2* 1000) 
l1s.append(l1)



In [ ]:
l1sum = 0.0
l2sum = 0.0
for i in range(0,len(l1s)):
    l1 = np.array(l1s[i])
    l2 = np.array(l2s[i])
    
    l1sum += l1
    l2sum += l2

In [ ]:
sum(l1sum+l2sum)  / 1e12 / 100 * 2700   * .51 * 2.3

In [ ]:
sum(l1)

In [ ]:
.003/1000 * 510e6 * 2400 / 1e9 * 1e6 * .3

In [ ]:
1.32647322e+10 / 3.77065868e+11

In [ ]:
l1s = []
l2s = []

In [ ]:
@jit()
def deprecate():
    ny, nx = np.shape(Z[0,:,:])
    print(ny)
    fact = 100
    z = np.zeros((int(ny/fact)+1,int(nx/fact)+1))
    for i in range(0,ny,fact):
        for j in range(0,nx,fact):
            z[int(i/fact)+1,int(j/fact)+1] = Z[0,i,j]
    return z
        

In [ ]:
plt.imshow(deprecate())

In [ ]:
## Asia

#fluvial
l1 = [50774996120.465225,
 47282695412.81846,
 81329288596.249,
 46632212468.130905,
 16247242524.561226]


#diffusion
l2 = [53849736042.111496,
 34622039732.69555,
 44429869448.106125,
 12390034902.692486,
 12954441805.785233]
l2s.append(l2)
l1s.append(l1)

In [ ]:
#NA

#fluvial
l1 = [22090045018.558407,
 16126997023.134878,
 24445508476.27974,
 1207827295.7407196,
 1484109.9649295697]

l1s.append(l1)

#diffusion
l2 = [18540757.828004587,
 9293283.953959662,
 17025284.73091651,
 1894050.1666102153,
 9302.58162506381]
l2 = [i *1000 for i in l2]
l2s.append(l2)


In [ ]:
#sa

#fluvial
l1 = [18489542526.118042,
 13786719955.694822,
 26727212146.923477,
 15720076445.341122,
 2289920794.616355]
l1s.append(l1)

l2=[45329695001.572014,
 8889907081.115767,
 1540985993.1576538,
 4653457.266656535,
 0.0]
l2s.append(l2)


In [ ]:
sum(l1)

In [ ]:
# au

l1 = [15869198290.863245,
 974767130.7095572,
 23314114.5912808,
 1432380.3893487994,
 110450399.76283571]
l1s.append(l1)


l2=[11929201.178884065,
 2679935.1682861596,
 671427.3535527467,
 1881.049626855462,
 0.0]
l2 = [i *1000 for i in l2]
l2s.append(l2)
np.sum(l1)

In [ ]:
#af

l1 = [26294106173.025497,
 23609149813.36735,
 16762050050.967857,
 629058151.8146291,
 3678634.4955713795]
l1s.append(l1)

l2 = [30591097.097434204,
 22037383.424389534,
 17132479.507036813,
 654879.1994445655,
 6537.980181630266]
l2 = [i *1000 for i in l2]

l2s.append(l2)


In [ ]:
sum(l1) /77013471868

In [ ]:
sum(l2)

In [ ]:
l1sum = 0.0
l2sum = 0.0
for i in range(0,len(l1s)):
    l1 = np.array(l1s[i])
    l2 = np.array(l2s[i])
    l1sum += l1
    l2sum += l2

In [ ]:
l2sum

In [ ]:
np.sum(l1sum+l2sum) / 1e9 / 100 * 2400 / 1000 

In [ ]:
import matplotlib
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 62}
matplotlib.rc('font', **font)
plt.figure(figsize=(15,10))

plt.bar(range(0,3),(l1sum+l2sum)[[0,2,3]]/ 1e12 / 100 * 2700   * .51 * 2.3)
plt.ylabel('Erosion rate (GT/Yr)')
plt.xlabel('Local slope')
plt.xticks([0,1,2])
ax = plt.gca()

ax.set_xticklabels(['0.01-.1','0.1-1','>1'])
# ax2 = ax.twinx()
# ax2.bar(range(3),l1sum[[0,1,2]],alpha=.5,color='r')

#plt.yscale('log')

In [ ]:
plt.bar()

In [ ]:
np.sum([53849736042.111496,
 34622039732.69555,
 44429869448.106125,
 12390034902.692486,
 12954441805.785233]) *2400 *.5 /1e9 /1000 / 100 * 3

In [ ]:
plt.figure(figsize=(7,5))

plt.ylabel('Erosion contribution (Gt))')
plt.xlabel('Elev range (m)')
plt.bar(['0-','1000-2500','2500-4000','4000+'],l


In [ ]:
l[l<0] = 0

In [ ]:
np.save('/Volumes/T7/E',E)

In [ ]:
np.save('/Volumes/T7/E',E)

In [ ]:
import geopandas as gpd